In [1]:
import pandas as pd
import numpy as np
import datetime as dt

# Load GTFS Data

In [2]:
agency = pd.read_csv('GTFS/amtrak/agency.txt')
calendar = pd.read_csv('GTFS/amtrak/calendar.txt')
routes = pd.read_csv('GTFS/amtrak/routes.txt')
stop_times = pd.read_csv('GTFS/amtrak/stop_times.txt')
stops = pd.read_csv('GTFS/amtrak/stops.txt')
transfers = pd.read_csv('GTFS/amtrak/transfers.txt')
trips = pd.read_csv('GTFS/amtrak/trips.txt')

# Merge Tables

Match service_id to trip_id

In [3]:
service_trip_id = pd.merge(calendar[['service_id']],trips[['service_id', 'trip_id']])
service_trip_id.head()

,service_id,trip_id
0,409714,2100409714
1,409716,2100409716
2,409718,2100409718
3,433850,2103433850
4,433851,2103433851


Merge calendar with trip_id

In [4]:
calendar = pd.merge(calendar,service_trip_id)
calendar.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,trip_id
0,409714,1,1,1,1,1,0,0,20130124,20130215,2100409714
1,409716,1,1,1,1,1,0,0,20130219,20130524,2100409716
2,409718,1,1,1,1,1,0,0,20130528,20140528,2100409718
3,433850,1,1,1,1,1,0,0,20130303,20130322,2103433850
4,433851,1,1,1,0,0,0,0,20130325,20130327,2103433851


# Define functions

Here's what the stop_times table looks like:

In [5]:
stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,pickup_type,drop_off_type
0,2100409714,6:00:00,6:00:00,WAS,1,0,0
1,2100409714,6:21:00,6:21:00,BWI,2,0,0
2,2100409714,6:32:00,6:34:00,BAL,3,0,0
3,2100409714,7:16:00,7:16:00,WIL,4,0,0
4,2100409714,7:33:00,7:35:00,PHL,5,0,0


Function to add arrival dates to stop_times tables. Deals with arrival times > 24 hours.

In [24]:
def add_arrival_date(df, dates=['1/4/2016', '1/5/2016', '1/6/2016','1/7/2016']):
    df = df.copy()
    arrival_date = []
    arrival_time = []

    for i in df['arrival_time']:
        hour = i[:i.find(':')]
        minute = i[i.find(':')+1:i.find(':',4)]
        second = i[i.find(':',5)+1:]
        
        if int(hour) < 24:
            arrival_date.append(dates[0])
            arrival_time.append(i)
        elif 24 <= int(hour) < 48:
            arrival_date.append(dates[1])
            hour = int(hour) - 24
            arrival_time.append(str(hour)+":"+minute+":"+second)
        elif 48 <= int(hour) < 72:
            arrival_date.append(dates[2])
            hour = int(hour) - 48
            arrival_time.append(str(hour)+":"+minute+":"+second)
        elif 72 <= int(hour) < 96:
            arrival_date.append(dates[3])
            hour = int(hour) - 72
            arrival_time.append(str(hour)+":"+minute+":"+second)
        else:
            arrival_date.append('NA')
            
    df['arrival_date'] = arrival_date
    df['arrival_time'] = arrival_time
    return df

Function to add departure dates to stop_times tables. Deals with departure times >24 hours.

In [25]:
def add_departure_date(df, dates=['1/4/2016', '1/5/2016', '1/6/2016','1/7/2016']):
    df = df.copy()
    departure_date = []
    departure_time = []

    for i in df['departure_time']:
        hour = i[:i.find(':')]
        minute = i[i.find(':')+1:i.find(':',4)]
        second = i[i.find(':',5)+1:]
        
        if int(hour) < 24:
            departure_date.append(dates[0])
            departure_time.append(i)
        elif 24 <= int(hour) < 48:
            departure_date.append(dates[1])
            hour = int(hour) - 24
            departure_time.append(str(hour)+":"+minute+":"+second)
        elif 48 <= int(hour) < 72:
            departure_date.append(dates[2])
            hour = int(hour) - 48
            departure_time.append(str(hour)+":"+minute+":"+second)
        elif 72 <= int(hour) < 96:
            departure_date.append(dates[3])
            hour = int(hour) - 72
            departure_time.append(str(hour)+":"+minute+":"+second)
        else:
            departure_date.append('NA')
            
    df['departure_date'] = departure_date
    df['departure_time'] = departure_time
    return df

# Get timetables for each day of the week

In [26]:
monday = calendar[calendar['monday'] == 1]
tuesday = calendar[calendar['tuesday'] == 1]
wednesday = calendar[calendar['wednesday'] == 1]
thursday = calendar[calendar['thursday'] == 1]
friday = calendar[calendar['friday'] == 1]
saturday = calendar[calendar['saturday'] == 1]
sunday = calendar[calendar['sunday'] == 1]

Store the results here:

In [27]:
legslist = []

## Monday

In [28]:
monday.head()

,service_id,monday,tuesday,wednesday,thursday,friday,saturday,sunday,start_date,end_date,trip_id
0,409714,1,1,1,1,1,0,0,20130124,20130215,2100409714
1,409716,1,1,1,1,1,0,0,20130219,20130524,2100409716
2,409718,1,1,1,1,1,0,0,20130528,20140528,2100409718
3,433850,1,1,1,1,1,0,0,20130303,20130322,2103433850
4,433851,1,1,1,0,0,0,0,20130325,20130327,2103433851


In [33]:
monday_trips = monday['trip_id']
print len(monday_trips)
monday_trips.head()

1114


0    2100409714
1    2100409716
2    2100409718
3    2103433850
4    2103433851
Name: trip_id, dtype: int64

In [30]:
for i in monday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/4/2016', '1/5/2016', '1/6/2016','1/7/2016'])
    df = add_departure_date(df, dates=['1/4/2016', '1/5/2016', '1/6/2016','1/7/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

## Tuesday

In [31]:
tuesday_trips = tuesday['trip_id']
tuesday_trips.head()
print len(tuesday_trips)

916


In [32]:
for i in tuesday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/5/2016', '1/6/2016', '1/7/2016','1/8/2016'])
    df = add_departure_date(df, dates=['1/5/2016', '1/6/2016', '1/7/2016','1/8/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

## Wednesday

In [36]:
wednesday_trips = wednesday['trip_id']
wednesday_trips.head()
print len(wednesday_trips)

917


In [37]:
for i in wednesday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/6/2016', '1/7/2016', '1/8/2016','1/9/2016'])
    df = add_departure_date(df, dates=['1/6/2016', '1/7/2016', '1/8/2016','1/9/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

## Thursday

In [38]:
thursday_trips = thursday['trip_id']
thursday_trips.head()
print len(thursday_trips)

888


In [39]:
for i in thursday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/7/2016', '1/8/2016', '1/9/2016','1/10/2016'])
    df = add_departure_date(df, dates=['1/7/2016', '1/8/2016', '1/9/2016','1/10/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

## Friday

In [40]:
friday_trips = friday['trip_id']
friday_trips.head()
print len(friday_trips)

882


In [41]:
for i in friday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/8/2016', '1/9/2016', '1/10/2016','1/11/2016'])
    df = add_departure_date(df, dates=['1/8/2016', '1/9/2016', '1/10/2016','1/11/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

## Saturday

In [42]:
saturday_trips = saturday['trip_id']
saturday_trips.head()
print len(saturday_trips)

575


In [43]:
for i in saturday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/9/2016', '1/10/2016', '1/11/2016','1/12/2016'])
    df = add_departure_date(df, dates=['1/9/2016', '1/10/2016', '1/11/2016','1/12/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

## Sunday

In [44]:
sunday_trips = sunday['trip_id']
sunday_trips.head()
print len(sunday_trips)

700


In [45]:
for i in sunday_trips:
    df = stop_times[stop_times['trip_id'] == i]
    df = add_arrival_date(df, dates=['1/10/2016', '1/11/2016', '1/12/2016','1/13/2016'])
    df = add_departure_date(df, dates=['1/10/2016', '1/11/2016', '1/12/2016','1/13/2016'])
    
    df['arrival_datetime'] = pd.to_datetime(df['arrival_date'] + ' ' + df['arrival_time'])
    df['departure_datetime'] = pd.to_datetime(df['departure_date'] + ' ' + df['departure_time'])


    # join df with stops
    df = pd.merge(df, stops[['stop_id', 'stop_name', 'stop_lat', 'stop_lon']],left_on='stop_id', right_on='stop_id')

    # join df with trips to get direction id
    df = pd.merge(df, trips[['trip_id', 'trip_short_name', 'direction_id', 'route_id']], left_on='trip_id', right_on='trip_id', how='left')

    # join df with routes to gt train name
    df = pd.merge(df, routes[['route_id', 'route_long_name']], left_on='route_id', right_on='route_id', how='left')

    # make a new dataframe called legs
    legs = pd.DataFrame()
    legs['starttime'] = df['departure_datetime']
    legs['stoptime'] = df['arrival_datetime'].shift(-1).fillna(method='ffill')
    legs['tripduration'] = (legs['stoptime'] - legs['starttime'])/np.timedelta64(1, 's')
    legs['tripduration'] = [int(x) for x in legs['tripduration']]
    legs['start station id'] = df['stop_id']
    legs['start station name'] = df['stop_name']
    legs['start station latitude'] = df['stop_lat']
    legs['start station longitude'] = df['stop_lon']
    legs['end station id'] = legs['start station id'].shift(-1).fillna(method='ffill')
    legs['end station name'] = legs['start station name'].shift(-1).fillna(method='ffill')
    legs['end station latitude'] = legs['start station latitude'].shift(-1).fillna(method='ffill')
    legs['end station longitude'] = legs['start station longitude'].shift(-1).fillna(method='ffill')
    legs['route_id'] = 1 # I'm choosing to set American trains to route_id = 1
    legs['direction'] = df['direction_id']
    legs['trainName'] = df['route_long_name']
    legs['trainNumber'] = df['trip_short_name']

    legslist.append(legs)

# View the aggregate data

In [46]:
data = pd.concat(legslist)

In [47]:
len(data)

88019

In [50]:
data.tail()

,starttime,stoptime,tripduration,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,route_id,direction,trainName,trainNumber
10,2016-01-10 22:03:00,2016-01-10 22:24:00,1260,NLS,Niles Amtrak,41.837412,-86.252372,NBU,New Buffalo Amtrak,41.796656,-86.745782,1,0,Wolverine,355
11,2016-01-10 22:24:00,2016-01-10 22:34:00,600,NBU,New Buffalo Amtrak,41.796656,-86.745782,MCI,Michigan City Amtrak,41.721103,-86.905385,1,0,Wolverine,355
12,2016-01-10 22:34:00,2016-01-10 23:08:00,2040,MCI,Michigan City Amtrak,41.721103,-86.905385,HMI,Hammond-Whiting Amtrak,41.691155,-87.506511,1,0,Wolverine,355
13,2016-01-10 23:08:00,2016-01-10 23:57:00,2940,HMI,Hammond-Whiting Amtrak,41.691155,-87.506511,CHI,Chicago Union Station,41.878684,-87.639443,1,0,Wolverine,355
14,2016-01-10 23:57:00,2016-01-10 23:57:00,0,CHI,Chicago Union Station,41.878684,-87.639443,CHI,Chicago Union Station,41.878684,-87.639443,1,0,Wolverine,355


# Print to csv

Remember to delete index column before running processing script.

In [51]:
data.to_csv('./amtrak/legsconcat2.csv')

This worked pretty well!

I manually deleted a few train legs:

Autotrain 52 and 53 between Lorton VA and Sanford FL. 

Empire Builder from Seattle to St Paul.

This is an excel formula to calculate geodesic distance: =ACOS(COS(RADIANS(90-F2)) *COS(RADIANS(90-J2)) +SIN(RADIANS(90-F2)) *SIN(RADIANS(90-J2)) *COS(RADIANS(G2-K2))) *6371

# Color coding by train name

In [59]:
np.unique(data['trainName'])

array(['Acela Express', 'Adirondack', 'Auto Train', 'Blue Water',
       'California Zephyr', 'Capitol Corridor', 'Capitol Limited',
       'Cardinal', 'Carl Sandburg', 'Carolinian', 'City Of New Orleans',
       'Coast Starlight', 'Crescent', 'Downeaster', 'Empire Builder',
       'Empire Service', 'Ethan Allen Express', 'Heartland Flyer',
       'Hiawatha Service', 'Hoosier State', 'Illini', 'Illinois Zephyr',
       'Keystone Service', 'Lake Shore Limited', 'Lincoln Service',
       'Missouri River Runner', 'Northeast Regional', 'Pacific Surfliner',
       'Palmetto', 'Pennsylvanian', 'Pere Marquette', 'Piedmont', 'Saluki',
       'San Joaquin', 'Shuttle', 'Silver Meteor', 'Silver Star',
       'Southwest Chief', 'Texas Eagle', 'Vermonter', 'Wolverine'], dtype=object)

In [71]:
colors = pd.read_csv('amtrak/colors.csv')
colors

,Region,Train,RegionNumber,TrainNumber
0,Northeast Corridor,Acela Express,1,1
1,Northeast Corridor,Northeast Regional,1,2
2,Northeast Corridor,Downeaster,1,3
3,Northeast Corridor,Keystone Service,1,4
4,Northeast Corridor,Empire Service,1,5
5,East,Adirondack,2,6
6,East,Vermonter,2,7
7,East,Ethan Allen Express,2,8
8,East,Auto Train,2,9
9,East,Silver Meteor,2,10


Join data to color scheme, matching on train name

In [74]:
data_colors = pd.merge(data, colors, left_on='trainName', right_on='Train')
data_colors.head()

,starttime,stoptime,tripduration,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,route_id,direction,trainName,trainNumber,Region,Train,RegionNumber,TrainNumber
0,2016-01-04 06:00:00,2016-01-04 06:21:00,1260,WAS,Washington Union Station,38.896993,-77.006422,BWI,BWI Marshall Airport Amtrak,39.192362,-76.694300,1,0,Acela Express,2100,Northeast Corridor,Acela Express,1,1
1,2016-01-04 06:21:00,2016-01-04 06:32:00,660,BWI,BWI Marshall Airport Amtrak,39.192362,-76.694300,BAL,Baltimore Penn Station,39.307302,-76.615688,1,0,Acela Express,2100,Northeast Corridor,Acela Express,1,1
2,2016-01-04 06:34:00,2016-01-04 07:16:00,2520,BAL,Baltimore Penn Station,39.307302,-76.615688,WIL,Wilmington Amtrak,39.737263,-75.551095,1,0,Acela Express,2100,Northeast Corridor,Acela Express,1,1
3,2016-01-04 07:16:00,2016-01-04 07:33:00,1020,WIL,Wilmington Amtrak,39.737263,-75.551095,PHL,Philadelphia 30th Street Station,39.955615,-75.181041,1,0,Acela Express,2100,Northeast Corridor,Acela Express,1,1
4,2016-01-04 07:35:00,2016-01-04 08:29:00,3240,PHL,Philadelphia 30th Street Station,39.955615,-75.181041,NWK,Newark Penn Station,40.734706,-74.164750,1,0,Acela Express,2100,Northeast Corridor,Acela Express,1,1


Save to csv

In [76]:
data_colors.to_csv("amtrak/legsconcat3.csv")